## Learning Language Models

### Learning about Attention - By Hand

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import gensim
from nltk.data import find
import nltk

nltk.download("word2vec_sample")

[nltk_data] Downloading package word2vec_sample to
[nltk_data]     C:\Users\I745352\AppData\Roaming\nltk_data...
[nltk_data]   Package word2vec_sample is already up-to-date!


True

In [2]:
def softmax(x, axis=0):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x)/np.sum(np.exp(x))

def get_word2vec_embedding(words):
    """Function that takes in a list of words and returns a list of their embeddings, 
    based on a pretrained word2vec encoder."""
    word2vec_sample = str(find("models/word2vec_sample/pruned.word2vec.txt"))
    model = gensim.models.KeyedVectors.load_word2vec_format(
        word2vec_sample, binary=False
    )
    
    output=[]
    words_pass = []
    for word in words:
        try:
            output.append(np.array(model.word_vec(word)))
            words_pass.append(word)
        except:
            pass
    
    embeddings = np.array(output)
    del model
    return embeddings, words_pass
    

### Cross Attention
![Attention in Detail](attention_in_detail.png "Attention in Detail")